In [30]:
from openai import OpenAI
from dotenv import load_dotenv
# OPEN AI API 키 설정
load_dotenv()

True

In [31]:
import os
import json

MY_API_KEY = os.environ['OPENAI_API_KEY']

In [32]:
client = OpenAI(api_key=MY_API_KEY)

### 1번 과제: 게시글 제목 다듬기

- 상황 : 대학교에서 교수님들과 대학생들에게 공개된 게시판을 운영하고 있는 운영자로써, 부적절한 제목의 게시글이 올라왔을 때 AI를 활용하여 감지 후, 부적절한 표현을 적절하게 수정한 뒤 게시글이 업로드되도록 하는 시스템을 개발하고자 함
- 제약사항
    1. 게시글이 올라오는 상황은 주어진 텍스트 파일을 한 줄씩 읽는 상황으로 가정
    2. 주어진 제목은 비속어가 있을 수도 있고 없을 수도 있음
    3. 게시글 내용은 고려하지 않음
- 코드 구현 시 참고사항
    1. 제목을 입력으로 받는 함수로 구현할 것
    2. 서비스에서 활용할 때 비속어가 있는지 없는지 확인이 가능해야함
    3. 비속어가 있는 경우 변환된 제목을 같이 받아와야함
- 프롬프트 구성 방식
    1. system: 배경상황 혹은 페르소나 설정 
    2. instruction: 사용자의 입력이 어떻게 들어오는지 명시, 비속어가 있는 경우, 없는 경우 처리 지시
    3. context
    4. input data: 입력값 (제목)
    4. output specification: 출력값의 형태를 지정

### 함수 구성

In [67]:
def check_title(title:str) -> dict:
    """
    제목에 비속어가 포함되어 있는지 확인하는 함수

    Args:
        title (str): 제목

    Returns:
        dict: 비속어 포함 여부와 수정된 제목
        key: bad_word, fixed_title
    """
    

    user_prompt = f'''
        당신은 대학교에서 교수님들과 대학생들에게 공개된 게시판을 운영하고 있는 운영자입니다.
        당신에게 주어진 게시판의 제목은 다음과 같습니다 : "{title}"
        비속어가 한 단어라도 포함될 시 해당 단어를 반드시 순화하여 수정하세요. 
        출력은 json형태로 마크다운 없이 반환하세요
  
        bad_word: "예", 혹은 "아니오"
        fixed_title: 올바른 단어로 수정된 내용
        '''
    
    response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )

    return json.loads(response.choices[0].message.content)

### 함수 실험

In [68]:
example = check_title('너 또라이니')

try:
    print('나쁜 말입니까?:',example['bad_word'])
    if example['bad_word'] == "예":
        print('수정된 내용:',example['fixed_title'])
except:
    pass

나쁜 말입니까?: 예
수정된 내용: 너 이상하니


### 출력

In [69]:
with open('title.txt', 'r', encoding='utf-8') as f:
    titles = f.readlines()

for title in titles:
    print('원래 제목: ', title)
    result = check_title(title)

    print('비속어 포함 여부: ', result['bad_word'])
    if result['bad_word'] == '예':
        print('수정된 제목: ', result['fixed_title'])
    print('-'*100)


원래 제목:  이딴 거 왜 배워야 돼, 씨발

비속어 포함 여부:  예
수정된 제목:  이딴 거 왜 배워야 돼, 정말
----------------------------------------------------------------------------------------------------
원래 제목:  개같은 과제 또 내냐

비속어 포함 여부:  예
수정된 제목:  힘든 과제 또 내냐
----------------------------------------------------------------------------------------------------
원래 제목:  수업 내용에 대해 질문이 있습니다

비속어 포함 여부:  아니오
----------------------------------------------------------------------------------------------------
원래 제목:  이런 병신 같은 규칙 좀 바꿔라

비속어 포함 여부:  예
수정된 제목:  이런 부적절한 규칙 좀 바꿔라
----------------------------------------------------------------------------------------------------
원래 제목:  과제 제출 기한 문의드립니다

비속어 포함 여부:  아니오
----------------------------------------------------------------------------------------------------
원래 제목:  게시판 이용 규칙 안내 부탁드립니다

비속어 포함 여부:  아니오
----------------------------------------------------------------------------------------------------
원래 제목:  강의 자료 추가 업로드 요청합니다

비속어 포함 여부:  아니오
--------------------------------

### 2번 과제: 영어 단어장 만들기

- 상황 : 영어 공부를 위해 영어 단어들을 제시하는 AI 서비스를 개발하려고 합니다. 단어장의 구성은 영단어, 한국어 의미, 유사어, 반의어, 예문 등을 포함해야 합니다.
- 제약사항
    1. 제안받는 단어의 난이도를 설정할 수 있어야합니다.
    2. 한번 제안받은 단어는 다시 제안받지 않도록 해야합니다.
- 코드 구현 시 참고사항
    1. 사용자의 입력에 따라 다음 단어를 제안받거나, 멈출 수 있어야합니다.
    2. 지금까지 제안받은 단어 목록을 지속적으로 관리할 수 있도록 클래스화, 파일관리를 활용합니다.

### 함수 구성

In [149]:
import json

class EnglishWordAI:

    def __init__(self, name: str, level: str):
        """
        Args:
            name: 서비스를 이용하는 사람의 이름\n
            level: 서비스를 이용하는 사람의 난이도
        """
        self.name = name
        self.level = level
        # 여기서 서비스를 이용하는 사람의 이름과 난이도를 전달받습니다.
        # 서비스를 이용하는 사람을 통해 파일에서 기존에 학습한 단어장이 있다면 불러옵니다.
        
        try:
            with open(f'{name}_{level}.txt', 'r', encoding='utf-8') as f:
                ex_words = f.readlines()
            self.ex_words = [ex_word.strip() for ex_word in ex_words]
        except:
            self.ex_words = []
        
        
        ...
    
    def save_words(self):
        """
        지금까지 학습한 단어장을 파일로 저장합니다.
        """
        
        with open(f'{self.name}_{self.level}.txt', 'w', encoding='utf-8') as f:
            for ex_word in self.ex_words:  
                print('한줄씩 쓰는중:',ex_word)
                f.write(f'{ex_word}\n')
        
        ...
    
    def get_word(self):
        """
        새로운 단어를 추천받습니다.
        """
        # 프롬프트를 통해 새로운 단어를 추천받습니다.  
        
        user_prompt = f'''
        영어 공부를 위해 영어 단어들을 제시하는 영어 선생님입니다. 
        난이도에 따라 단어장의 수준을 조절하세요. 주어진 난이도 : {self.level}
        단어장의 구성은 영단어, 한국어 의미, 유사어, 반의어, 예문 등을 포함해야 합니다.
        다음 목록에 있는 영단어는 제외하고 단어장에 새로 추가할 영단어를 1개 추천하세요.
        목록 : {self.ex_words}
        
        출력은 json형태로 마크다운 없이 반환하세요.
  
        'word' : 영단어
        'korean_meaning' : 한국어 의미
        'similar_word' : 유사어
        'antonym' : 반의어
        'example_sentence' : 예문
        '''
    
        response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )
        
        self.ex_words.append(json.loads(response.choices[0].message.content)['word'])
        
        return json.loads(response.choices[0].message.content)




### 함수 실험

In [150]:
ai = EnglishWordAI('seul', 'easy')
print(ai.ex_words)
ai.get_word()
ai.save_words()


['my', 'me', 'bye', 'hello', 'happy', 'friend', 'smile']
한줄씩 쓰는중: my
한줄씩 쓰는중: me
한줄씩 쓰는중: bye
한줄씩 쓰는중: hello
한줄씩 쓰는중: happy
한줄씩 쓰는중: friend
한줄씩 쓰는중: smile
한줄씩 쓰는중: book


### 출력

In [151]:
ai = EnglishWordAI('jason', 'easy')

while True:
    if input('continue? (y/n): ') == 'n':
        break
    else:
        word_dict = ai.get_word()
        print(f"단어: {word_dict['word']}, 뜻: {word_dict['korean_meaning']}")
        print(f"유사어: {word_dict['similar_word']}, 반의어: {word_dict['antonym']}")
        print(f"예문: {word_dict['example_sentence']}")

단어: happy, 뜻: 행복한
유사어: ['joyful', 'content', 'pleased'], 반의어: ['sad', 'unhappy', 'miserable']
예문: She felt happy when she received the good news.
단어: joyful, 뜻: 기쁜
유사어: ['cheerful', 'delighted'], 반의어: ['sad', 'unhappy']
예문: She felt joyful when she heard the good news.
단어: cheerful, 뜻: 쾌활한
유사어: merry, 반의어: gloomy
예문: She always has a cheerful smile on her face.


In [152]:
print(ai.ex_words)

['happy', 'joyful', 'cheerful']


### 3번 과제: 면접 질문 생성하기

- 상황 : 서류에 합격한 뒤, 면접을 준비하기 위해 예상 면접 질문을 AI를 활용해 제안받으려고 합니다.  
면접 질문은 채용 공고와 작성한 자기소개서를 기반으로 추천받을 수 있어야하며, 예상 질문 뒤에 본인의 답변이 적절했는지 AI를 활용해 검토할 수 있어야 합니다.

In [168]:
def get_interview_question(recruit_str, cover_letter_str,ex_question):
    
        user_prompt = f'''
        당신은 채용공고와 지원자의 자기소개서를 바탕으로 질문하는 인사담당자입니다. 
        주어진 채용공고와 자기소개서를 바탕으로 면접 예상 질문 1개를 작성하세요.
        다음 목록에 있는 질문은 제외하고 새로운 질문을 1개 작성하세요.
        목록 : {ex_question}
        
        채용공고 : {recruit_str}
        자기소개서 : {cover_letter_str}
        
        출력은 json형태로 마크다운 없이 반환하세요.
        'interview_question' : 면접 예상 질문
        '''
    
        response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )
        print(json.loads(response.choices[0].message.content))
        return json.loads(response.choices[0].message.content)
    
def evaluate_answer(interview_question,my_answer):
        user_prompt = f'''
        당신은 인사담당자의 질문과 지원자의 대답이 적절한지 판별하는 인사담당자입니다. 
        주어진 질문과 지원자의 대답을 바탕으로 지원자의 답변의 적절성을 1~100점으로 점수로 평가하고 그 이유를 한 줄로 서술하세요.
        인사담당자의 질문 : {interview_question}
        지원자의 대답 : {my_answer}
        
        출력은 json형태로 마크다운 없이 반환하세요.
        'score' : 지원자 대답의 점수 (1~100)
        'reason' : 그 이유
        '''
    
        response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )
        print('질문자의 답변:',my_answer)
        print(json.loads(response.choices[0].message.content))
        
        return json.loads(response.choices[0].message.content)
    

In [ ]:
recruit_str = '마케팅 기획팀 (프로모션 업무 담당) 팀원 (신입~대리급)'
cover_letter_str = '저는 경영학과를 졸업하였습니다'
ex_question = []


while True:
    if input('continue? (y/n): ') == 'n':
        break
    else:
        interview_question = get_interview_question(recruit_str, cover_letter_str,ex_question)
        ex_question.append(interview_question)

        my_answer = input('면접자의 답변 입력')
        evaluate_answer(interview_question,my_answer) 

{'interview_question': '경영학과에서 배운 지식을 마케팅 기획과 프로모션 업무에 어떻게 적용할 수 있을지 설명해 주시겠습니까?'}
질문자의 답변: 그냥 뽑아주세요
{'score': 5, 'reason': '지원자의 대답은 질문에 대한 구체적인 설명이나 관련성을 전혀 제공하지 않으며, 성의가 부족합니다.'}
{'interview_question': '경영학과에서 배운 지식을 활용하여 마케팅 기획팀에서의 팀워크를 어떻게 강화할 수 있을지 설명해 주시겠습니까?'}
질문자의 답변: 저 팀워크 좋아요
{'score': 20, 'reason': '지원자의 대답은 질문에 대한 구체적인 설명이나 경영학 지식의 활용 방안이 부족하여 적절하지 않습니다.'}


## 실전 1. 게시판 관리 자동화

- 상황 : 하루에 1000건 정도 게시글이 올라오는 취업준비생 카페를 관리하는 운영자로써 작성되는 게시물의 종류를 구분하고, 작성되는 게시물에 따라 적절하게 반응하는 자동화 AI 서비스를 개발하고자 합니다.

- 제약사항
    1. 게시물은 제목없이 250자의 내용만으로 구성되어있습니다.
    2. 게시물의 종류는 홍보 글, 고민 상담 글, 신규 채용공고 글, 합격 후기 글 등이 있습니다.

- 응용분야 : 고객 문의 관리 자동화

## 구상

**구현목록**
- 게시글 random생성 : 게시글 id / 작성자 id / 게시글 내용
- 비속어 : 게시글 내용은 순화하기 / 작성자 id에게는 경고하기
- 입력된 게시글에 대해 카테고리 배정하기 / 입력되는 스팸 처리하기
- 등록된 게시글에 대해 들어오는 신고(ex>불법홍보,불안감 조성) 처리하기

**프로세스**
- 게시글 입력 -> 비속어/스팸 탐지 & 카테고리 지정 -> 게시글 등록 -> 등록된 게시글에 들어오는 신고 확인

**llm배치**
1. 게시글 생성 gpt : 게시글 랜덤생성(작성자 id , 게시글 내용)

2. 게시글 판별 gpt : 비속어/스팸 탐지 & 카테고리 지정 

3. 문의 판별 gpt : 문의의 카테고리를 '신고'와 '단순문의'로 분류
    - 신고일 경우> 
        - 게시글 색출 gpt : 문의와 가장 유사한 게시글 색출
        - 신고 판별 gpt : 게시글과 문의가 타당한지 확인
    - 단순 문의일 경우> 답변 gpt : 아는 내에서 간단하게 자유 답변

## 게시글 목록 정의

In [ ]:
uploaded_post_dict = {}
trash_post_ls = []
report_ls = []
to_be_remove_id_ls = []
post_id = 0

In [313]:
def create_post_gpt(post_role,post_describe):
        user_prompt = f'''
        당신은 취업준비생 카페에 게시글을 작성하는 작성자입니다. 주어지는 작성자 역할과 게시글의 설명에 따라 게시글 내용을 250자 이내로 1개 생성하세요.
        작성자 역할 : {post_role}
        게시글의 설명 : {post_describe}
        작성자 5자 이내의 한글 + 4자리 숫자로 임의로 1개 생성하세요.
        
        출력은 반드시 json형태로 마크다운 없이 반환하세요.
        'writer_id' : 작성자 id
        'post_content' : 게시글 내용
        '''
    
        response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )
        return json.loads(response.choices[0].message.content)
    
    
def classify_post_gpt(created_post_content):
        user_prompt = f'''
        당신은 취업준비생 카페에 올라오는 게시글을 감독하는 관리자입니다. 
        올라온 게시글이 스팸 혹은 혐오감 조성 게시글인지 판단하고 게시글의 카테고리를 분류하세요.하나의 게시글에는 하나의 카테고리만 존재합니다.
        올라온 게시글 : {created_post_content}
        카테고리 종류 : 홍보, 고민 상담, 신규 채용 공고, 합격 후기, 삭제 예정
        
        출력은 반드시 json형태로 마크다운 없이 반환하세요.
        'category' : 카테고리 종류
        '''
    
        response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )
        return json.loads(response.choices[0].message.content)
    
    
def classify_query_gpt(query_input):
        user_prompt = f'''
        당신은 취업준비생 카페에 올라오는 게시글에 대한 문의의 카테고리를 판별하는 관리자입니다. 하나의 게시글에는 하나의 카테고리만 존재합니다.
        문의 : {query_input}
        카테고리 종류 : 신고, 단순 문의
        
        출력은 반드시 json형태로 마크다운 없이 반환하세요.
        'category' : 카테고리 종류
        '''
    
        response = client.chat.completions.create(model = "gpt-4o",
                                          messages = [
                                              {'role': 'user', 'content': user_prompt},
                                              ],
                                          max_tokens=150,
                                          temperature=0
                                         )
        return json.loads(response.choices[0].message.content)
    
def find_post_gpt(created_query,uploaded_post_dict):
    user_prompt = f'''
        당신은 취업준비생 카페에 올라오는 게시글에 대한 문의와 가장 관련이 깊은 게시글을 1개 찾아내는 관리자입니다.
        문의 : {created_query}
        게시글 목록 : {uploaded_post_dict}
        
        찾아낸 dict 자료형의 key값만 마크다운 없이 반환하세요. 
        '''
    response = client.chat.completions.create(model = "gpt-4o",
                                        messages = [
                                            {'role': 'user', 'content': user_prompt},
                                            ],
                                        max_tokens=150,
                                        temperature=0
                                        )
    return int(response.choices[0].message.content)

def check_query_with_post_gpt(query_input,reported_post):
    user_prompt = f'''
        당신은 취업준비생 카페에 올라온 특정 게시글과 해당 게시글에 대한 신고 문의가 타당한지 판별하는 관리자입니다.
        해당 게시글이 삭제해야할 대상인지 yes 혹은 no로 판별하고 그 이유를 한줄로 서술하세요.
        
        신고 문의 : {query_input}
        게시글 : {reported_post}
        
        출력은 반드시 json형태로 마크다운 없이 반환하세요. 작성자 id와 게시글 내용을 변경하는 것은 엄격히 금지합니다.
        'go_trash' : yes or no
        'writer_id' : 작성자 id
        'post_content': 게시글 내용
        'manager_answer' : 이유
        '''
    response = client.chat.completions.create(model = "gpt-4o",
                                        messages = [
                                            {'role': 'user', 'content': user_prompt},
                                            ],
                                        max_tokens=150,
                                        temperature=0
                                        )
    return json.loads(response.choices[0].message.content)

def answer_to_query_gpt(query_input):
    user_prompt = f'''
        당신은 취업준비생 카페에 올라온 문의에 대해 답변하는 관리자입니다. 해당 문의에 대해 한줄로 답변하세요.
        문의 : {query_input}
        
        출력은 반드시 json형태로 마크다운 없이 반환하세요. 작성자 id와 게시글 내용을 변경하는 것은 엄격히 금지합니다.
        'query_content': 문의
        'manager_answer' : 답변
        '''
    response = client.chat.completions.create(model = "gpt-4o",
                                        messages = [
                                            {'role': 'user', 'content': user_prompt},
                                            ],
                                        max_tokens=150,
                                        temperature=0
                                        )
    return json.loads(response.choices[0].message.content)


## 함수 실행

In [ ]:
##게시글 생성하기
created_post = create_post_gpt('홍보','쉽게 돈을 버는 방법') # (역할, 생성할 게시글 설명) / 오류 날 가능성 높음
created_post_content = created_post['post_content']
## 게시글 판별하기
created_post['category'] = classify_post_gpt(created_post_content)['category']
created_post['warning'] = 0

print('생성된 post:',created_post)

## 카테고리 판별하기
if created_post['category'] == '삭제 예정':
    created_post['warning'] += 1
    trash_post_ls.append(created_post)
else:
    uploaded_post_dict[post_id]=created_post
    post_id += 1


생성된 post: {'writer_id': '홍보1234', 'post_content': '안녕하세요, 여러분! 요즘 취업 준비로 바쁘시죠? 그래서 준비했습니다! 쉽게 돈을 버는 방법을 소개합니다. 시간과 장소에 구애받지 않고, 스마트폰만 있으면 누구나 가능합니다. 관심 있으신 분들은 댓글로 문의 주세요!', 'category': '삭제 예정', 'warning': 0}


In [302]:
uploaded_post_dict

{0: {'writer_id': '인사담1234',
  'post_content': '안녕하세요, 인사담당자입니다. 저희 회사에서 신규 채용을 진행합니다. 다양한 분야에서 열정적이고 책임감 있는 인재를 모집합니다. 많은 지원 부탁드리며, 자세한 내용은 홈페이지를 참고해주세요. 감사합니다!',
  'category': '신규 채용 공고',
  'warning': 0},
 1: {'writer_id': '홍보1234',
  'post_content': '안녕하세요, 취업 준비에 도움이 될 만한 사이트를 하나 소개해드릴게요. 다양한 취업 정보와 면접 팁을 제공하는 곳인데요, 실제로 많은 취준생들이 이곳을 통해 좋은 결과를 얻었다고 하네요. 관심 있으신 분들은 한번 방문해보세요!',
  'category': '홍보',
  'warning': 0},
 2: {'writer_id': '인사담1234',
  'post_content': '안녕하세요, 취업준비생 여러분. 여러분의 노력을 잘 알고 있습니다. 하지만, 준비가 부족한 상태로 지원하는 경우가 많습니다. 자기소개서와 면접 준비에 더 많은 시간을 투자하세요. 작은 차이가 큰 결과를 만듭니다. 더 나은 준비로 좋은 결과 얻길 바랍니다.',
  'category': '고민 상담',
  'warning': 0}}

In [314]:
## 문의 분류하기
query_input = input('문의를 입력하세요')
created_query= classify_query_gpt(query_input)
created_query['query'] = query_input

if created_query['category'] == "신고":
    reported_post_id = find_post_gpt(created_query,uploaded_post_dict)
    reported_post = uploaded_post_dict[reported_post_id]
    checked_reported_post = check_query_with_post_gpt(query_input,reported_post)
    if checked_reported_post['go_trash'] == 'yes':
        print('삭제 대상 게시글: ',checked_reported_post)
        to_be_remove_id_ls.append(reported_post_id)
    report_ls.append(checked_reported_post)

else:
    checked_reported_post = answer_to_query_gpt(query_input)
    report_ls.append(checked_reported_post)
    
print(report_ls[-1])

{'query_content': '안녕하세요', 'manager_answer': '안녕하세요, 무엇을 도와드릴까요?'}


In [311]:
print(uploaded_post_dict)
print(trash_post_ls)
print(report_ls)
print(to_be_remove_id_ls) # uploaded_post_dict의 key값만 있으므로 삭제 가능

{0: {'writer_id': '인사담1234', 'post_content': '안녕하세요, 인사담당자입니다. 저희 회사에서 신규 채용을 진행합니다. 다양한 분야에서 열정적이고 책임감 있는 인재를 모집합니다. 많은 지원 부탁드리며, 자세한 내용은 홈페이지를 참고해주세요. 감사합니다!', 'category': '신규 채용 공고', 'warning': 0}, 1: {'writer_id': '홍보1234', 'post_content': '안녕하세요, 취업 준비에 도움이 될 만한 사이트를 하나 소개해드릴게요. 다양한 취업 정보와 면접 팁을 제공하는 곳인데요, 실제로 많은 취준생들이 이곳을 통해 좋은 결과를 얻었다고 하네요. 관심 있으신 분들은 한번 방문해보세요!', 'category': '홍보', 'warning': 0}, 2: {'writer_id': '인사담1234', 'post_content': '안녕하세요, 취업준비생 여러분. 여러분의 노력을 잘 알고 있습니다. 하지만, 준비가 부족한 상태로 지원하는 경우가 많습니다. 자기소개서와 면접 준비에 더 많은 시간을 투자하세요. 작은 차이가 큰 결과를 만듭니다. 더 나은 준비로 좋은 결과 얻길 바랍니다.', 'category': '고민 상담', 'warning': 0}}
[{'writer_id': '홍보1234', 'post_content': '안녕하세요, 여러분! 요즘 취업 준비로 바쁘시죠? 그래서 준비했습니다! 쉽게 돈을 버는 방법을 소개합니다. 시간과 장소에 구애받지 않고, 스마트폰만 있으면 누구나 가능합니다. 관심 있으신 분들은 댓글로 문의 주세요!', 'category': '삭제 예정', 'warning': 1}]
[{'go_trash': 'yes', 'writer_id': '홍보1234', 'post_content': '안녕하세요, 취업 준비에 도움이 될 만한 사이트를 하나 소개해드릴게요. 다양한 취업 정보와 면접 팁을 제공하는 곳인데요, 실제로 많은 취준생들이 이곳을 통해 좋은 결과를 얻었다고 하네요. 

## 실전 2. 주가 정보 수집 스크래퍼 제작

- 상황 : [업종별 시세](https://finance.naver.com/sise/sise_group.naver?type=upjong)에서 업종별로 전일대비 얼마나 상승, 하락했는지를 특정 시간에 받아볼 수 있도록 자동화된 시스템을 개발하려고 합니다.

In [27]:
from bs4 import BeautifulSoup
import requests

response = requests.get('https://finance.naver.com/sise/sise_group.naver?type=upjong')
soup = BeautifulSoup(response.text, 'html.parser')
tbody = soup.select('div#contentarea_left table tr')

tb_dict = {}
for tr in tbody[3:]:
    td_ls = tr.select('td')
    if len(td_ls) > 2:
        tb_dict[td_ls[0].text.strip()] = td_ls[1].text.strip()
tb_dict

{'담배': '+5.42%',
 '레저용장비와제품': '+3.68%',
 '기계': '+3.40%',
 '건축자재': '+3.37%',
 '에너지장비및서비스': '+3.36%',
 '전기유틸리티': '+3.29%',
 '판매업체': '+3.15%',
 '방송과엔터테인먼트': '+3.10%',
 '생물공학': '+2.78%',
 '창업투자': '+2.43%',
 '전기장비': '+2.41%',
 '전자장비와기기': '+2.40%',
 '생명과학도구및서비스': '+2.28%',
 '가구': '+2.09%',
 '양방향미디어와서비스': '+1.94%',
 '호텔,레스토랑,레저': '+1.93%',
 '인터넷과카탈로그소매': '+1.85%',
 '건강관리기술': '+1.72%',
 '백화점과일반상점': '+1.67%',
 '증권': '+1.65%',
 '출판': '+1.63%',
 '음료': '+1.61%',
 '은행': '+1.48%',
 '무선통신서비스': '+1.46%',
 '제약': '+1.45%',
 '포장재': '+1.42%',
 '도로와철도운송': '+1.42%',
 '건강관리업체및서비스': '+1.38%',
 '식품과기본식료품소매': '+1.36%',
 '광고': '+1.31%',
 '컴퓨터와주변기기': '+1.31%',
 '건강관리장비와용품': '+1.27%',
 '핸드셋': '+1.21%',
 '부동산': '+1.20%',
 '교육서비스': '+1.18%',
 '전자제품': '+1.15%',
 'IT서비스': '+1.13%',
 '자동차': '+1.13%',
 '건설': '+1.11%',
 '게임엔터테인먼트': '+1.10%',
 '디스플레이장비및부품': '+1.09%',
 '카드': '+1.02%',
 '복합유틸리티': '+0.96%',
 '화장품': '+0.94%',
 '자동차부품': '+0.89%',
 '반도체와반도체장비': '+0.87%',
 '섬유,의류,신발,호화품': '+0.86%',
 '가정용기기와용품': '+0.85%',
 '통신장비'

In [42]:
def recommand_industry_gpt(tb_dict):
    user_prompt = f'''
        당신은 실력있는 증권가 입니다. 주어지는 정보는 업종과 전일대비 상승률로 이루어져 있습니다. 이를 기반으로 업종명을 추천하고 그 이유를 한줄로 서술하세요. 
        정보 : {tb_dict}
        
        출력은 반드시 json형태로 마크다운 없이 반환하세요. 업종명을 변경하는 것은 엄격히 금지합니다.
        'recommand_industry': 업종명
        'recommand_reason' : 이유
        '''
    response = client.chat.completions.create(model = "gpt-4o",
                                        messages = [
                                            {'role': 'user', 'content': user_prompt},
                                            ],
                                        max_tokens=150,
                                        temperature=0
                                        )
    return json.loads(response.choices[0].message.content)

In [43]:
recommand_industry_gpt(tb_dict)

{'recommand_industry': '담배',
 'recommand_reason': '담배 업종이 전일대비 가장 높은 상승률인 +5.42%를 기록하여 투자 매력이 높습니다.'}